# Bitcoin Daily Prices & Log Returns

This notebook:

1. Downloads historical daily BTCUSDT prices from Binance using `Binance.load_bitcon_1d`.
2. Stores the result in a pandas DataFrame.
3. Computes daily log returns.
4. Computes the cumulative average (mean to date) of daily log returns.
5. Plots price and return series with Plotly.

Adjust the date range below as needed.

In [29]:
# Imports and date range
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots  # <-- added

from polymarket_analysis.data.binance import Binance

# Parameters
DAYS_BACK = 300  # change as needed
END = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
START = END - timedelta(days=DAYS_BACK)

# Plot cutoff (inclusive) - stop plot in May 2025
PLOT_END_DATE = datetime(2025, 5, 31, tzinfo=timezone.utc).date()

print(f"Fetching daily BTCUSDT data from {START.date()} to {END.date()} ({DAYS_BACK} days)")
print(f"Plot will be truncated at {PLOT_END_DATE}")

Fetching daily BTCUSDT data from 2024-10-18 to 2025-08-14 (300 days)
Plot will be truncated at 2025-05-31


In [30]:
# Download data
df = Binance.load_bitcon_1d(START, END)

# Keep relevant columns, ensure proper types
df = df[['timestamp','price']].copy()
df = df.sort_values('timestamp')

# Convert timestamp to date (UTC) for daily grouping
df['date'] = df['timestamp'].dt.date

# Daily close price (since interval is 1d we already have 1 row per day)
daily = df[['date','price']].set_index('date')
print(daily.head())
print(f"Loaded {len(daily)} daily rows")

               price
date                
2024-10-18  68428.00
2024-10-19  68378.00
2024-10-20  69031.99
2024-10-21  67377.50
2024-10-22  67426.00
Loaded 301 daily rows


In [31]:
# Compute daily log returns
# log return r_t = ln(P_t / P_{t-1})
daily['log_return'] = np.log(daily['price'] / daily['price'].shift(1))

# Drop first NaN
returns = daily.dropna().copy()

# Forward-looking average log return from each day until the final day (today)
# NOTE: This is an ex-post metric (not knowable on that past date in real time).
returns['avg_log_return_forward'] = returns['log_return'][::-1].expanding().mean()[::-1]

# 30-day rolling average (backward-looking) of daily log returns
# Uses a simple moving average over the last 30 observations (approx 30 calendar days)
returns['rolling_30d_log_return'] = returns['log_return'].rolling(30).mean()

returns.tail()

,price,log_return,avg_log_return_forward,rolling_30d_log_return
date,,,,
2025-08-10,119294.01,0.024024,0.002769,0.000497
2025-08-11,118686.00,-0.005110,-0.002545,0.000357
2025-08-12,120134.08,0.012127,-0.001690,0.000292
2025-08-13,123306.43,0.026064,-0.008598,0.000950
2025-08-14,118085.85,-0.043261,-0.043261,0.000093


In [32]:
# Plot with Plotly (price + forward average, truncated to May 2025)
plot_returns = returns.loc[returns.index <= PLOT_END_DATE]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.04,
                    subplot_titles=("BTC Daily Close Price", "Avg Forward & 30D Rolling Log Return") )

# Row 1: Price
fig.add_trace(go.Scatter(x=plot_returns.index, y=plot_returns['price'], name='Price', line=dict(color='#1f77b4')), row=1, col=1)

# Row 2: Forward average log return
fig.add_trace(go.Scatter(x=plot_returns.index, y=plot_returns['avg_log_return_forward'], name='Forward Avg Log Return', line=dict(color='#2ca02c')), row=2, col=1)

# Row 2: 30-day rolling average
fig.add_trace(go.Scatter(x=plot_returns.index, y=plot_returns['rolling_30d_log_return'], name='30D Rolling Log Return', line=dict(color='#d62728', dash='dash')), row=2, col=1)

fig.update_layout(height=650, title='BTC Daily Price & Log Return Averages (through May 2025)', showlegend=True, legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
fig.update_yaxes(title_text='Price (USDT)', row=1, col=1)
fig.update_yaxes(title_text='Log Return', row=2, col=1)
fig.show(renderer="browser")